In [1]:
import time
import numpy as np
import tensorflow as tf
from tensorflow import keras

print(tf.__version__)

2.0.0-beta1


In [2]:
def make_toy_dataset(num_data):
    X = np.random.randn(num_data, 3)
    y = 3 * X[:, 0] - 2 * X[:, 1]**3 + 2 * X[:, 2]**2 + 0.5 * np.random.randn(num_data)
    y = y[:, np.newaxis]
    return X, y

In [3]:
def build_model():
    model = keras.models.Sequential([
        keras.layers.Dense(10, activation="relu", input_dim=3),
        keras.layers.Dense(10, activation="relu"),
        keras.layers.Dense(1)
    ])

    return model

In [4]:
N_train = 40000
N_test = 4000

num_epochs = 10
learning_rate = 0.01
batch_size = 64

In [5]:
# get toy datasets
X_train, y_train = make_toy_dataset(N_train)
X_test, y_test = make_toy_dataset(N_test)

# make datasets
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=512).batch(batch_size)

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_dataset = test_dataset.batch(100)

In [6]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)

model.compile(optimizer, loss="mse", metrics=["mae"])

start = time.time()
model.fit(X_train, y_train, batch_size=batch_size, epochs=num_epochs, validation_data=(X_test, y_test))
end = time.time()

print(end - start, "(sec)")

Train on 40000 samples, validate on 4000 samples
Epoch 1/10
40000/40000 [==============================] - 1s 34us/sample - loss: 20.8785 - mae: 2.0121 - val_loss: 23.1281 - val_mae: 2.3853
Epoch 2/10
40000/40000 [==============================] - 1s 28us/sample - loss: 21.4569 - mae: 2.0702 - val_loss: 17.4709 - val_mae: 1.8779
Epoch 3/10
40000/40000 [==============================] - 1s 29us/sample - loss: 20.1491 - mae: 2.2558 - val_loss: 16.9986 - val_mae: 2.1039
Epoch 4/10
40000/40000 [==============================] - 1s 27us/sample - loss: 20.2106 - mae: 2.3894 - val_loss: 21.4594 - val_mae: 2.7616
Epoch 5/10
40000/40000 [==============================] - 1s 26us/sample - loss: 22.0698 - mae: 2.6033 - val_loss: 16.0861 - val_mae: 1.7284
Epoch 6/10
40000/40000 [==============================] - 1s 26us/sample - loss: 20.7067 - mae: 2.4448 - val_loss: 20.8101 - val_mae: 2.9489
Epoch 7/10
40000/40000 [==============================] - 1s 27us/sample - loss: 21.4307 - mae: 2.5867 - 

In [7]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()

start = time.time()

for epoch in range(1, num_epochs+1):
    
    # train step 
    for x_batch, y_batch in train_dataset:
        with tf.GradientTape() as tape:
            y_pred = model(x_batch)
            loss = loss_fn(y_batch, y_pred)
        grads = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)
    
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    for x_batch, y_batch in test_dataset:
        y_pred = model(x_batch)
        loss = loss_fn(y_batch, y_pred)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))

end = time.time()
print(end - start, "(sec)")

epoch 1: loss=19.388, val_loss=6.017464, mae=2.058, val_mae=1.298
epoch 2: loss=14.959, val_loss=20.358269, mae=1.681, val_mae=2.008
epoch 3: loss=21.478, val_loss=12.725346, mae=2.173, val_mae=1.874
epoch 4: loss=16.453, val_loss=18.775377, mae=2.232, val_mae=1.906
epoch 5: loss=18.049, val_loss=21.427860, mae=2.272, val_mae=1.969
epoch 6: loss=24.390, val_loss=17.955822, mae=2.391, val_mae=1.749
epoch 7: loss=15.706, val_loss=4.267157, mae=2.237, val_mae=1.111
epoch 8: loss=9.485, val_loss=6.076662, mae=1.701, val_mae=1.599
epoch 9: loss=8.370, val_loss=9.201537, mae=1.599, val_mae=2.159
epoch 10: loss=9.432, val_loss=6.011655, mae=1.684, val_mae=1.645
67.33499646186829 (sec)


In [8]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()


@tf.function
def train_step(x, y):
    #print("tracing:", model, optimizer, x, y)
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return y_pred, loss

@tf.function
def eval_step(x, y):
    #print("tracing:", model, x, y)
    y_pred = model(x)
    loss = loss_fn(y, y_pred)
    return y_pred, loss
    
start = time.time()
    
for epoch in range(1, num_epochs+1):
    
    # train step 
    for x_batch, y_batch in train_dataset:
        
        y_pred, loss = train_step(x_batch, y_batch)
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)
    
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    for x_batch, y_batch in test_dataset:
        y_pred, loss = eval_step(x_batch, y_batch)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))
    
end = time.time()
print(end - start, "(sec)")

epoch 1: loss=18.941, val_loss=7.673271, mae=2.099, val_mae=1.595
epoch 2: loss=18.722, val_loss=13.874675, mae=2.145, val_mae=2.391
epoch 3: loss=19.804, val_loss=26.229481, mae=2.227, val_mae=2.639
epoch 4: loss=46.824, val_loss=72.437012, mae=3.544, val_mae=5.147
epoch 5: loss=77.826, val_loss=72.400139, mae=5.236, val_mae=5.151
epoch 6: loss=77.826, val_loss=72.400139, mae=5.236, val_mae=5.151
epoch 7: loss=77.826, val_loss=72.400139, mae=5.236, val_mae=5.151
epoch 8: loss=77.826, val_loss=72.400139, mae=5.236, val_mae=5.151
epoch 9: loss=77.826, val_loss=72.400139, mae=5.236, val_mae=5.151
epoch 10: loss=77.826, val_loss=72.400139, mae=5.236, val_mae=5.151
24.06973624229431 (sec)


In [9]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()


@tf.function
def train_step(x, y):
    #print("tracing:", model, optimizer, x, y)
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return y_pred, loss

@tf.function
def train_epoch(train_dataset):
    for x_batch, y_batch in train_dataset:
        
        y_pred, loss = train_step(x_batch, y_batch)
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)


@tf.function
def eval_step(x, y):
    #print("tracing:", model, x, y)
    y_pred = model(x)
    loss = loss_fn(y, y_pred)
    return y_pred, loss
    
@tf.function
def eval_epoch(test_dataset):
    for x_batch, y_batch in test_dataset:
        y_pred, loss = eval_step(x_batch, y_batch)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)

    
start = time.time()
    
for epoch in range(1, num_epochs+1):
    
    # train step 
    train_epoch(train_dataset)
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    eval_epoch(test_dataset)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))
    
end = time.time()
print(end - start, "(sec)")

epoch 1: loss=22.800, val_loss=23.001825, mae=2.201, val_mae=2.252
epoch 2: loss=23.168, val_loss=17.122089, mae=2.206, val_mae=1.905
epoch 3: loss=21.588, val_loss=17.114124, mae=2.224, val_mae=1.894
epoch 4: loss=19.981, val_loss=17.360416, mae=2.229, val_mae=2.223
epoch 5: loss=20.611, val_loss=18.949278, mae=2.427, val_mae=2.594
epoch 6: loss=21.235, val_loss=17.074144, mae=2.559, val_mae=2.316
epoch 7: loss=21.006, val_loss=16.820246, mae=2.539, val_mae=2.287
epoch 8: loss=21.351, val_loss=15.526962, mae=2.550, val_mae=1.986
epoch 9: loss=21.109, val_loss=16.922367, mae=2.574, val_mae=2.299
epoch 10: loss=21.178, val_loss=15.903707, mae=2.541, val_mae=2.035
8.744144678115845 (sec)


In [10]:
model = build_model()
optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
loss_fn = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean()
test_loss = keras.metrics.Mean()
train_mae_metric = keras.metrics.MeanAbsoluteError()
test_mae_metric = keras.metrics.MeanAbsoluteError()


def train_step(x, y):
    #print("tracing:", model, optimizer, x, y)
    with tf.GradientTape() as tape:
        y_pred = model(x)
        loss = loss_fn(y, y_pred)
    grads = tape.gradient(loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return y_pred, loss

@tf.function
def train_epoch(train_dataset):
    for x_batch, y_batch in train_dataset:
        
        y_pred, loss = train_step(x_batch, y_batch)
        
        # accumulate metrics of one batch
        train_loss(loss)
        train_mae_metric(y_batch, y_pred)



def eval_step(x, y):
    #print("tracing:", model, x, y)
    y_pred = model(x)
    loss = loss_fn(y, y_pred)
    return y_pred, loss
    
@tf.function
def eval_epoch(test_dataset):
    for x_batch, y_batch in test_dataset:
        y_pred, loss = eval_step(x_batch, y_batch)
        
        # accumulate metrics
        test_loss(loss)
        test_mae_metric(y_batch, y_pred)

    
start = time.time()
    
for epoch in range(1, num_epochs+1):
    
    # train step 
    train_epoch(train_dataset)
    
    # get metrics of one epoch and reset their states
    train_loss_result = train_loss.result()
    train_loss.reset_states()
    
    train_mae_result = train_mae_metric.result()
    train_mae_metric.reset_states()
    
    
    # evaluation step
    eval_epoch(test_dataset)
    
    
    # get metrics and reset their states
    test_loss_result = test_loss.result()
    test_loss.reset_states()
    
    test_mae_result = test_mae_metric.result()
    test_mae_metric.reset_states()
    
    
    # print train logs
    template = "epoch {}: loss={:.3f}, val_loss={:3f}, mae={:.3f}, val_mae={:.3f}"
    print(template.format(epoch, train_loss_result, test_loss_result, train_mae_result, test_mae_result))
    
end = time.time()
print(end - start, "(sec)")

epoch 1: loss=22.421, val_loss=22.303837, mae=2.165, val_mae=2.090
epoch 2: loss=20.282, val_loss=13.559036, mae=1.821, val_mae=1.310
epoch 3: loss=13.936, val_loss=10.135809, mae=1.358, val_mae=1.027
epoch 4: loss=11.856, val_loss=20.876116, mae=1.396, val_mae=3.415
epoch 5: loss=10.750, val_loss=21.758427, mae=1.409, val_mae=3.590
epoch 6: loss=10.198, val_loss=18.974026, mae=1.455, val_mae=3.294
epoch 7: loss=10.099, val_loss=18.623226, mae=1.532, val_mae=3.284
epoch 8: loss=8.204, val_loss=10.179640, mae=1.250, val_mae=1.997
epoch 9: loss=7.060, val_loss=9.346371, mae=1.064, val_mae=1.900
epoch 10: loss=6.661, val_loss=8.727838, mae=1.043, val_mae=1.840
7.924725532531738 (sec)
